# Fetching Internet Pages

## What is URL?
Universal Resouce Locator
- scheme: protocol
- netloc: TCP connection 만들기 위한 server의 hostname(또는 IP 주소)와 port 번호를 알 수 있다.
- path: server 내의 path (path가 반드시 server내에 존재하는 파일일 필요는 없다. Web application이 이를 받아 특정 function을 call하여 수행하고 그 결과로서 html 파일을 생성할 수도 있다. - dynamic page)
- query string: Web application에 path와 함께 전달된다. Web application은 이것을 input parameter로 생각하여 작업을 수행한다.


In [ ]:
from urllib.parse import urlparse, parse_qs, urlencode

url = 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EC%BD%94%EB%A1%9C%EB%82%98+%EB%B0%94%EC%9D%B4%EB%9F%AC%EC%8A%A4'
r = urlparse(url)
print(r)
print(r.query)

Query string의 decoding과 Python dict로 변환

In [ ]:
qs_dict = parse_qs(r.query)   # query sting to dict
print(qs_dict)

URL 표현방식에 맞춘 encoding

In [ ]:
urlencode(qs_dict)

## HTTP Protocol
![HTTP request/response](https://lh3.googleusercontent.com/P0nLbMawFwlLE2qXdIBbSnxWYEaE9M3ARDmKVB1puAEFIIOVyzIWtafQVcbo8VHFfcn5Fnn8r0-Y)

## `urllib` standard library

In [ ]:
from urllib.request import urlopen, urlretrieve

f = urlopen(url)
print(f)
print(f.status)
print(f.headers['content-type'])

HTTP response는 다음 part로 나눠진다.
- status line
- header lines
- content

Status code 200은 request에 대해 성공적으로 회신한다는 의미다. 
Content가 html text 파일이고 encoding 방식 기술되었으니, 읽어서 UTF-8에서 unicode로 decode해야 함을 알 수 있다. (Web contents는 통상 **UTF-8**으로 encoding되어 있다.)

In [ ]:
text = f.read().decode('UTF-8')
print(text)

참고: Chrome browser에서 fetch한 web contents를 확인할 수 있다. 
>    `도구 더보기` >> `개발자 도구`

In [ ]:
import re

image_srcs = re.findall(r'<img.*?src="(.*?)".*?>', text)
len(image_srcs)

### urlretrieve - retrieve URL resource into local file
가져온 HTML content를 보면, 이 page를 완성하기 위해 여러개의 image를 download해야 함을 알 수 있다. 

urlretrieve funtion은 주어진 url의 content를 (decoding없이) 그대로 가져와 temporary file에 저장하고, file name과 response의 headers를 return한다.

저장할 file을 지정하려면, `urlretrieve(url, file_name)`

In [ ]:
file_name, headers = urlretrieve(image_srcs[0])
print(file_name)
print(headers)

In [ ]:
with open(file, 'rb') as f:  # open in binary mode
    print(f.read())

----
# `requests` library
`urlopen`과 비슷한 `requests` module도 많이 사용되고 있다. Standard module이 아니기 때문에 이 모듈을 쓰기 위해서는 설치해야 한다. (Anaconda package에는 이미 설치되어 있다.)

명령창에서
- `$ pip install requests`

참고: Requests: HTTP for Humans https://requests.readthedocs.io/en/master/

In [ ]:
import requests

# send GET method and receive response message
response = requests.get('https://search.naver.com/search.naver',
                       params={'query': '코로나 바이러스'})    
# response is an class instance
print(response)
print(response.status_code)
print(bool(response))         # successful including 200, 204, 304, ...

In [ ]:
# response.content            # body of response message (bytes)
print(response.encoding)      # guess the encoding based on response's headers
print(response.text[:200])    # decoded text

In [ ]:
print(response.headers)    # headers of response message

In [ ]:
response.request.headers

### Handling JSON response

In [ ]:
import requests

# Search GitHub's repositories for requests
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
)

print(response.headers['content-type'])
print(response.text[:200])

In [ ]:
# Inspect some attributes of the `requests` repository
json_response = response.json()     # convert json content to Python 
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')  # Python 3.6+
print(f'Repository description: {repository["description"]}')  # Python 3.6+

### HTTP POST method
참고: https://httbin.org site는 requests module 저자가 시험용으로 만든 site다. 여러가지 HTTP method들에 대해, 받은 request message를 json 등의 내용으로 회신해 준다.

In [ ]:
r = requests.post('https://httpbin.org/post', data={'key': 'value'})
print(r.headers)
if r:
    print(r.text)
    if 'application/json' in r.headers['content-type']:
        json_r = r.json()
        print(json_r)

In [ ]:
print(r.request.url)
print(r.request.headers)
print(r.request.body)

### Authorization and Session

In [ ]:
import requests
from getpass import getpass

with requests.Session() as session:
    session.auth = ('username', getpass())

    # Instead of requests.get(), you'll use session.get()
    response = session.get('https://api.github.com/user')

# You can inspect the response just like you did before
print(response.headers)
print(response.json())